In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

Mounted at /content/drive


In [ ]:
!pip install torch
!pip install torchvision
!pip install segmentation-models-pytorch
!pip install pycocotools
!pip install transformers
# !pip install wandb
# !pip install colorama
# !pip install ttach

!pip install kaggle
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/kaggle_competitions/kaggle.json /root/.kaggle  # kaggl.json위치 지정
!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=d798a06bb8d208a9c8a4d92f7aa0a1e1e18a01c838156ac6f12398878d875ada
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=6ee49e9f58c3b4fe89abf4a937981e9847df62e59489c1173dd88d9671bd0d9a
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c6

In [ ]:
!pip3 install openmim
!mim install mmengine
!mim install "mmcv>=2.0.0,<2.1.0"

!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

import mmdet
print(mmdet.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.1 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 25.7 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 5.4 MB/s eta 0:00:00
Cloning into 'mmdetection'...
remote: Enumerating objects: 36774, done.
remote: Counting objects: 100% (264/264), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 36774 (delta 95), reused 209 (delta 86),

In [ ]:
import os
import sys
import gc
import re
import time
import datetime
import numpy as np

import pandas as pd
import random
import glob
import zlib
import base64
from datetime import datetime
from tqdm.notebook import tqdm
import json
import itertools
from itertools import chain
from types import SimpleNamespace
import cv2
import PIL
from PIL import Image
# import wandb

import matplotlib.pyplot as plt
import seaborn as sns

import albumentations as A
from albumentations.pytorch import ToTensorV2
# import ttach as tta

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda import amp
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

import torchvision
import segmentation_models_pytorch as smp
from sklearn.model_selection import KFold, StratifiedKFold

import pycocotools
import pycocotools.mask as mask_util
from pycocotools.coco import COCO

# package
sys.path.append("/content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/input/detection_utils/")
import utils
from engine import train_one_epoch, evaluate

# color font
# from colorama import Fore, Back, Style
# r_ = Fore.RED
# b_ = Fore.BLUE
# c_ = Fore.CYAN
# g_ = Fore.GREEN
# y_ = Fore.YELLOW
# m_ = Fore.MAGENTA
# sr_ = Style.RESET_ALL

In [ ]:
%cd /content/
!kaggle competitions download -c hubmap-hacking-the-human-vasculature
!unzip -qq /content/hubmap-hacking-the-human-vasculature.zip

%cd /content/
!cp /content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/input/COCO_Dataset.zip /content/
!unzip -qq /content/COCO_Dataset.zip

if not os.path.exists(f'/content/mmdet_wheel/'):
    os.mkdir( f'/content/mmdet_wheel/')

%cd /content/mmdet_wheel/
!cp /content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/input/mmdet_wheel.zip /content/mmdet_wheel/
!unzip -qq /content/mmdet_wheel/mmdet_wheel.zip

# %cd /content/

# !cp /content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/input/whole_512_coco_dataset.zip /content/
# !unzip -qq /content/whole_512_coco_dataset.zip



/content
100% 3.96G/3.97G [00:44<00:00, 164MB/s]
100% 3.97G/3.97G [00:44<00:00, 96.5MB/s]
/content
/content/mmdet_wheel


In [ ]:
from mmengine.config import Config
base="mask-rcnn_x101-64x4d_fpn_2x_coco"
img_size = 1024
seed = 42
colab_v = "colabv13"
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from mmdet.apis import init_detector, inference_detector
config_file = '/content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/model/colabv12_mask-rcnn_x101-64x4d_fpn_2x_coco_1024/colabv12_custom_config_mask-rcnn_x101-64x4d_fpn_2x_coco_1024_1024.py'
checkpoint_file = '/content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/model/colabv12_mask-rcnn_x101-64x4d_fpn_2x_coco_1024/best_coco_segm_mAP_epoch_18.pth'


In [ ]:
%%writefile /content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/{colab_v}_custom_config_{base}_{img_size}_{img_size}.py

colab_v='colabv13'
base=f"{colab_v}_mask-rcnn_x101-64x4d_fpn_2x_coco"
dataset_type = 'CocoDataset'
classes = ('blood_vessel',)
data_root = '/content'
num_classes = 1
img_size = 1024
batch_size = 4
seed = 42

model = dict(
    type='MaskRCNN',
    data_preprocessor=dict(
        type='DetDataPreprocessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        pad_mask=True,
        pad_size_divisor=32),
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained',
                      checkpoint='torchvision://resnet101')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    roi_head=dict(
        type='StandardRoIHead',
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=dict(
            type='Shared2FCBBoxHead',
            in_channels=256,
            fc_out_channels=1024,
            roi_feat_size=7,
            num_classes=1,
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0, 0.0, 0.0, 0.0],
                target_stds=[0.1, 0.1, 0.2, 0.2]),
            reg_class_agnostic=False,
            loss_cls=dict(
                type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
        mask_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=14, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        mask_head=dict(
            type='FCNMaskHead',
            num_convs=4,
            in_channels=256,
            conv_out_channels=256,
            num_classes=1,
            loss_mask=dict(
                type='CrossEntropyLoss', use_mask=True, loss_weight=1.0))),
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=-1,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_across_levels=False,
            nms_pre=2000,
            nms_post=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.6,
                neg_iou_thr=0.6,
                min_pos_iou=0.6,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=512,
                pos_fraction=0.25,
                neg_pos_ub=-1,
                add_gt_as_proposals=True),
            mask_size=28,
            pos_weight=-1,
            debug=False)),
    test_cfg=dict(
        rpn=dict(
            nms_across_levels=False,
            nms_pre=1000,
            nms_post=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            score_thr=0.05,
            nms=dict(type='nms', iou_thr=0.6),
            max_per_img=100,
            mask_thr_binary=0.5)))

dataset_type = 'CocoDataset'
data_root = '/content'
backend_args = None
metainfo = dict(classes=classes, palette=[(255, 0, 0)])

train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(
        type='LoadAnnotations', with_bbox=True, with_mask=True,
        poly2mask=True),

    dict(type='Resize', scale=(img_size, img_size), keep_ratio=True),
    dict(type='RandomFlip', direction=['horizontal', 'vertical'], prob=0.6),
    dict(type='RandomShift',prob=0.1),
    dict(type='PhotoMetricDistortion', brightness_delta=16,  contrast_range=(0.7, 1.3),  saturation_range=(0.7, 1.3), hue_delta=16),
    dict(type='Normalize', mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True),
    dict(type='PackDetInputs'),
    # dict(type='DefaultFormatBundle'),
    # dict(type='Collect', keys=['images', 'bbox', 'segmentation'])
]

train_dataloader = dict(
    batch_size=4,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type='CocoDataset',
        # type='MultiImageMixDataset',
        metainfo=dict(classes=classes, palette=[(255, 0, 0)]),
        data_root='/content',
        ann_file='/content/coco_annotations_train_all_folds5_fold2.json',
        data_prefix=dict(
            img='/content/train/'),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=train_pipeline,
        backend_args=None))
val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',
        metainfo=dict(classes=classes, palette=[(255, 0, 0)]),
        data_root='/content',
        ann_file='/content/coco_annotations_valid_all_folds5_fold2.json',
        data_prefix=dict(
            img='/content/train/'),
        test_mode=True,
        pipeline=[
            dict(type='LoadImageFromFile', backend_args=None),
            dict(type='Resize', scale=(img_size, img_size), keep_ratio=True),
            dict(type='Normalize', mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True),
            dict(
                type='PackDetInputs',
                meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                           'scale_factor'))
        ],
        backend_args=None))
test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',
        metainfo=dict(classes=classes, palette=[(255, 0, 0)]),
        data_root='/content',
        ann_file='/content/coco_annotations_valid_all_folds5_fold2.json',
        data_prefix=dict(
            img='/content/train/'),
        test_mode=True,
        pipeline=[
            dict(type='LoadImageFromFile', backend_args=None),
            dict(type='Resize', scale=(img_size, img_size), keep_ratio=True),
            dict(type='Normalize', mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True),

            dict(
                type='PackDetInputs',
                meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                           'scale_factor'))
        ],
        backend_args=None))
val_evaluator = dict(
    type='CocoMetric',
    ann_file='/content/coco_annotations_valid_all_folds5_fold2.json',
    metric=['segm'],
    format_only=False,
    backend_args=None,
    iou_thrs=[0.6])
test_evaluator = dict(
    type='CocoMetric',
    ann_file='/content/coco_annotations_valid_all_folds5_fold2.json',
    metric=['segm'],
    format_only=False,
    backend_args=None,
    iou_thrs=[0.6])
optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=8e-5,weight_decay=1e-2,eps=1e-7),
    )

optimizer_config = dict(grad_clip=dict(max_norm=100, norm_type=2))
param_scheduler = [
    dict(
        type='LinearLR', start_factor=1e-5, by_epoch=False, begin=0, end=500),
    dict(
        type='MultiStepLR',
        by_epoch=True,
        begin=0,
        end=30,
        milestones=[5, 10, 15, 20, 25, 30],
        gamma=0.85)
]
default_hooks = dict(
    timer=dict(type='IterTimerHook'),
    logger=dict(type='LoggerHook', interval=200),
    param_scheduler=dict(type='ParamSchedulerHook'),
    checkpoint=dict(
        type='CheckpointHook', interval=1, save_best='coco/segm_mAP'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    visualization=dict(type='DetVisualizationHook'))

custom_hooks = [
    dict(
        type='EarlyStoppingHook',
        monitor='coco/segm_mAP',
        rule='greater',
        strict=False,
        check_finite=True,
        patience=15,
        stopping_threshold=None)
]
default_scope = 'mmdet'
env_cfg = dict(
    cudnn_benchmark=False,
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),
    dist_cfg=dict(backend='nccl'))
vis_backends = [dict(type='LocalVisBackend')]
visualizer = dict(
    type='DetLocalVisualizer',
    vis_backends=[dict(type='LocalVisBackend')],
    name='visualizer')
log_config = dict(hooks=[
    dict(type='TextLoggerHook'),
    dict(
        type='MMDetWandbHook',
        init_kwargs=dict(project='mmdetection'),
        interval=10,
        log_checkpoint=True,
        log_checkpoint_metadata=True,
        num_eval_images=100,
        bbox_score_thr=0.3)
])
log_processor = dict(type='LogProcessor', window_size=50, by_epoch=True)
log_level = 'INFO'
load_from = '/content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/model/colabv12_mask-rcnn_x101-64x4d_fpn_2x_coco_1024/best_coco_segm_mAP_epoch_18.pth'
resume = True
resume_from = '/content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/model/colabv12_mask-rcnn_x101-64x4d_fpn_2x_coco_1024/best_coco_segm_mAP_epoch_18.pth'
train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=35, val_interval=1)
val_cfg = dict(type='ValLoop')
test_cfg = dict(type='TestLoop')
work_dir = f'/content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/model/{base}_{img_size}/'

Overwriting /content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/colabv13_custom_config_mask-rcnn_x101-64x4d_fpn_2x_coco_1024_1024.py


In [ ]:
config = f'/content/drive/MyDrive/kaggle_competitions/vision/HuBMAP_Hacking_the_Human_Vasculature_2023/{colab_v}_custom_config_{base}_{img_size}_{img_size}.py'

In [ ]:
def seed_everthing(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed_everthing(seed)

pd.set_option('display.max_columns', 200) ## 모든 열 출력
pd.set_option('display.max_rows', 200) ## 모든 행 출력

In [ ]:
!python /content/mmdetection/tools/train.py {config}

07/31 06:04:00 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]
    CUDA available: True
    numpy_random_seed: 1965631406
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_

In [ ]:
- Variable : employername_160M Description :  Employer's name.
- Variable : pmtamount_36A Description :  Tax deductions amount for credit bureau payments.
- Variable : processingdate_168D Description :  Date when the tax deduction is processed.